## 📝 Conclusões e Insights

### 🎯 Resultados Alcançados
1. **Modelo CNN Personalizado**: Treinado do zero com data augmentation
2. **Transfer Learning**: Aproveitamento de pesos pré-treinados do ImageNet  
3. **Métricas Completas**: Accuracy, Precision, Recall, Confusion Matrix, ROC-AUC

### 💡 Pontos-Chave
- Transfer Learning geralmente apresenta melhor desempenho com menos dados
- Data Augmentation é essencial para evitar overfitting
- Batch Normalization estabiliza o treinamento
- Early Stopping previne overfitting

### 🚀 Próximos Passos
- [ ] Fine-tuning do modelo pré-treinado
- [ ] Testar com outras arquiteturas (ResNet, EfficientNet)
- [ ] Deploy da aplicação com Gradio/FastAPI
- [ ] Implementar explicabilidade (Grad-CAM)
- [ ] Testar em produção com dados reais

In [ ]:
# Compare both models
if test_dir.exists():
    print("📊 Comparing CNN vs Transfer Learning Models\n")
    
    # Evaluate custom CNN
    test_loss_cnn, test_acc_cnn, _, _ = model.evaluate(test_generator, verbose=0)
    
    # Evaluate transfer learning
    test_loss_tl, test_acc_tl = model_tl.evaluate(test_generator, verbose=0)
    
    # Create comparison table
    comparison_df = pd.DataFrame({
        'Model': ['Custom CNN', 'Transfer Learning (MobileNetV2)'],
        'Test Loss': [f'{test_loss_cnn:.4f}', f'{test_loss_tl:.4f}'],
        'Test Accuracy': [f'{test_acc_cnn:.4f} ({test_acc_cnn*100:.2f}%)', 
                         f'{test_acc_tl:.4f} ({test_acc_tl*100:.2f}%)'],
        'Parameters': [f'{model.count_params():,}', f'{model_tl.count_params():,}']
    })
    
    print(comparison_df.to_string(index=False))
    
    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    models_names = ['Custom CNN', 'Transfer Learning']
    accuracies = [test_acc_cnn*100, test_acc_tl*100]
    losses = [test_loss_cnn, test_loss_tl]
    
    colors = ['#FF6B6B', '#4ECDC4']
    
    axes[0].bar(models_names, accuracies, color=colors)
    axes[0].set_ylabel('Accuracy (%)', fontsize=12)
    axes[0].set_title('Model Accuracy Comparison', fontweight='bold')
    axes[0].set_ylim([0, 100])
    for i, v in enumerate(accuracies):
        axes[0].text(i, v+2, f'{v:.2f}%', ha='center', fontweight='bold')
    
    axes[1].bar(models_names, losses, color=colors)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Model Loss Comparison', fontweight='bold')
    for i, v in enumerate(losses):
        axes[1].text(i, v+0.02, f'{v:.4f}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Train transfer learning model
if train_dir.exists():
    print("🚀 Training Transfer Learning Model...")
    
    # Use same callbacks
    callbacks_tl = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0),
        ModelCheckpoint(f'../models/transfer_learning_{timestamp}.h5', 
                       monitor='val_accuracy', save_best_only=True, verbose=0)
    ]
    
    # Train
    history_tl = model_tl.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        callbacks=callbacks_tl,
        verbose=1
    )
    
    print("✅ Transfer Learning training complete!")
else:
    print("⚠️  No training data available.")

In [ ]:
# Create Transfer Learning model using MobileNetV2
def create_transfer_learning_model(num_classes=2):
    """
    Create transfer learning model using MobileNetV2
    """
    # Load pre-trained MobileNetV2
    base_model = keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Create new model
    model_tl = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model_tl

# Create and compile transfer learning model
print("🔄 Creating Transfer Learning Model (MobileNetV2)...")
model_tl = create_transfer_learning_model()

model_tl.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Transfer Learning model created!")
print(f"Total parameters: {model_tl.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model_tl.trainable_weights]):,}")

## 8️⃣ Transfer Learning: Comparação com MobileNetV2

Implementando Transfer Learning usando um modelo pré-treinado para comparação.

In [ ]:
# Function to predict on single image
def predict_image(image_path, class_names=['cat', 'dog']):
    """Predict class for a single image"""
    
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img.resize((IMG_SIZE, IMG_SIZE))) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict
    predictions = model.predict(img_array, verbose=0)
    pred_idx = np.argmax(predictions[0])
    confidence = predictions[0][pred_idx]
    
    return class_names[pred_idx], confidence, predictions[0]

# Test on sample images
if test_dir.exists():
    test_images = []
    for class_dir in (test_dir / 'cats').iterdir():
        if class_dir.is_file():
            test_images.append(class_dir)
            break
    
    for class_dir in (test_dir / 'dogs').iterdir():
        if class_dir.is_file():
            test_images.append(class_dir)
            break
    
    if test_images:
        fig, axes = plt.subplots(1, len(test_images), figsize=(12, 5))
        if len(test_images) == 1:
            axes = [axes]
        
        for idx, img_path in enumerate(test_images):
            pred_class, confidence, all_preds = predict_image(img_path)
            
            img = Image.open(img_path)
            axes[idx].imshow(img)
            axes[idx].set_title(f'Prediction: {pred_class.upper()}\nConfidence: {confidence:.2%}',
                              fontweight='bold', fontsize=11)
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.show()

## 7️⃣ Make Predictions on New Images

In [ ]:
# Plot ROC Curves
if test_dir.exists():
    from sklearn.preprocessing import label_binarize
    
    # One-hot encode true labels
    true_labels_onehot = label_binarize(true_labels, classes=range(len(class_names)))
    
    plt.figure(figsize=(10, 8))
    
    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels_onehot[:, i], predictions[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, linewidth=2, label=f'{class_names[i]} (AUC = {roc_auc:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random')
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - Test Set', fontweight='bold', fontsize=14)
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Get predictions and create confusion matrix
if test_dir.exists():
    predictions = model.predict(test_generator)
    pred_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator.classes
    
    # Confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    
    # Classification report
    class_names = list(test_generator.class_indices.keys())
    report = classification_report(true_labels, pred_labels, target_names=class_names)
    
    print("\n📋 Classification Report:")
    print(report)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=class_names,
               yticklabels=class_names,
               cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix - Test Set', fontweight='bold', fontsize=14)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

In [ ]:
# Evaluate on test set
if test_dir.exists():
    test_generator = val_test_datagen.flow_from_directory(
        str(test_dir),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    print("📊 Evaluating on test set...")
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
    
    print("\n✅ Test Set Metrics:")
    print(f"  Loss: {test_loss:.4f}")
    print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"  Precision: {test_precision:.4f}")
    print(f"  Recall: {test_recall:.4f}")
else:
    print("⚠️  No test data found.")

## 6️⃣ Evaluate Model Performance

In [ ]:
# Plot training history
if 'history' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_title('Model Accuracy', fontweight='bold')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('Model Loss', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Validation', linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title('Model Precision', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Validation', linewidth=2)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].set_title('Model Recall', fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Train model
if train_dir.exists():
    print("🚀 Starting training...")
    print(f"Timestamp: {timestamp}")
    
    # Create data generators
    train_generator = train_datagen.flow_from_directory(
        str(train_dir),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    validation_generator = val_test_datagen.flow_from_directory(
        str(val_dir),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    print(f"\nClasses: {train_generator.class_indices}")
    print(f"Training samples: ~{len(train_generator) * BATCH_SIZE}")
    print(f"Validation samples: ~{len(validation_generator) * BATCH_SIZE}")
    
    # Train
    history = model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        callbacks=callbacks,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator)
    )
    
    print("\n✅ Training complete!")
else:
    print("⚠️  No training data found. Skipping training.")

In [ ]:
# Define callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        f'../models/cnn_classifier_{timestamp}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    TensorBoard(
        log_dir=f'../logs/{timestamp}',
        histogram_freq=1
    )
]

print("✅ Callbacks configured!")
print("- EarlyStopping: patience=10")
print("- ModelCheckpoint: Save best model")
print("- ReduceLROnPlateau: Reduce LR if val_loss plateaus")

In [ ]:
# Compile model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("✅ Model compiled!")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Loss: Categorical Crossentropy")
print(f"Metrics: Accuracy, Precision, Recall")

## 5️⃣ Compile and Train Model

In [ ]:
# Define CNN model architecture
def create_cnn_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=2):
    """
    Create a custom CNN model for image classification
    """
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', 
                     input_shape=input_shape, name='conv1_1'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool1'),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_1'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool2'),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_1'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool3'),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_1'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='conv4_2'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), name='pool4'),
        layers.Dropout(0.25),
        
        # Global Average Pooling
        layers.GlobalAveragePooling2D(),
        
        # Fully Connected Layers
        layers.Dense(512, activation='relu', name='fc1'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(256, activation='relu', name='fc2'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax', name='output')
    ])
    
    return model

# Create model
model = create_cnn_model()

# Display model architecture
print("🧠 Model Architecture:")
model.summary()

# Plot model architecture
from tensorflow.keras.utils import plot_model
try:
    plot_model(model, to_file='model_architecture.png', show_shapes=True)
    print("✅ Model architecture saved to 'model_architecture.png'")
except:
    print("visualization library not available")

## 4️⃣ Build CNN Model Architecture

Construindo uma CNN com:
- 4 blocos convolucionais (32, 64, 128, 256 filtros)
- Batch Normalization para estabilidade
- MaxPooling para reduzir dimensionalidade
- Dropout para prevenir overfitting
- Camadas totalmente conectadas

In [ ]:
# Configure image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Just rescaling for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

print("✅ Data generators created!")
print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")

# Show augmentation examples
if train_dir.exists():
    # Get sample image
    sample_cat = list((train_dir / 'cats').glob('*.jpg'))[0]
    sample_img = load_img(sample_cat, target_size=(IMG_SIZE, IMG_SIZE))
    
    # Plot augmented versions
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    for i, ax in enumerate(axes.ravel()):
        if i == 0:
            ax.imshow(sample_img)
            ax.set_title('Original', fontweight='bold')
        else:
            # Generate augmented image
            augmented_img = train_datagen.random_transform(np.array(sample_img))
            augmented_img = np.clip(augmented_img, 0, 255).astype(np.uint8)
            ax.imshow(augmented_img)
            ax.set_title(f'Augmented {i}', fontweight='bold')
        ax.axis('off')
    
    plt.suptitle('Data Augmentation Examples', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 3️⃣ Data Preprocessing and Augmentation

### Características das imagens
- Redimensionamento: 224x224 pixels (padrão ResNet)
- Normalização: Valores de pixel entre 0-1
- Augmentação: Rotação, flip, zoom, etc.

In [ ]:
# Visualize sample images
def visualize_sample_images(data_dir, num_images=4):
    """Visualize sample images from dataset"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    train_cat_dir = data_dir / 'train' / 'cats'
    train_dog_dir = data_dir / 'train' / 'dogs'
    
    if train_cat_dir.exists() and train_dog_dir.exists():
        cat_images = list(train_cat_dir.glob('*.jpg')) + list(train_cat_dir.glob('*.png'))
        dog_images = list(train_dog_dir.glob('*.jpg')) + list(train_dog_dir.glob('*.png'))
        
        # Show cats
        for i in range(2):
            if i < len(cat_images):
                img = Image.open(cat_images[i])
                axes[i].imshow(img)
                axes[i].set_title('Cat', fontsize=12, fontweight='bold')
                axes[i].axis('off')
        
        # Show dogs
        for i in range(2):
            if i < len(dog_images):
                img = Image.open(dog_images[i])
                axes[2+i].imshow(img)
                axes[2+i].set_title('Dog', fontsize=12, fontweight='bold')
                axes[2+i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Call visualization if data exists
if train_dir.exists():
    visualize_sample_images(data_dir)

In [ ]:
# Check data availability
train_dir = data_dir / 'train'
val_dir = data_dir / 'validation'
test_dir = data_dir / 'test'

print("📊 Dataset Summary:")
print(f"Train directory: {train_dir} - Exists: {train_dir.exists()}")
print(f"Validation directory: {val_dir} - Exists: {val_dir.exists()}")
print(f"Test directory: {test_dir} - Exists: {test_dir.exists()}")

# Count images if directories exist
def count_images_in_dir(directory):
    """Count images in directory and subdirectories"""
    if not directory.exists():
        return 0
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len([f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    return count

if train_dir.exists():
    train_count = count_images_in_dir(train_dir)
    val_count = count_images_in_dir(val_dir)
    test_count = count_images_in_dir(test_dir)
    
    print(f"\n📈 Image Counts:")
    print(f"Training images: {train_count}")
    print(f"Validation images: {val_count}")
    print(f"Test images: {test_count}")
    print(f"Total: {train_count + val_count + test_count}")
else:
    print("\n⚠️  Dataset not found. Please download and organize the Cats vs Dogs dataset.")
    print("Expected structure:")
    print("data/")
    print("├── train/")
    print("│   ├── cats/")
    print("│   └── dogs/")
    print("├── validation/")
    print("│   ├── cats/")
    print("│   └── dogs/")
    print("└── test/")
    print("    ├── cats/")
    print("    └── dogs/")

In [ ]:
# Download Microsoft Cats vs Dogs dataset
import urllib.request
import tarfile

# Create data directory
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

# Download dataset (note: this is a large file ~786MB)
print("⏳ Downloading Cats vs Dogs dataset...")
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

# Alternative: Use smaller sample or local files
# For demonstration, we'll assume data is already organized
print("📁 Data directory structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(str(data_dir), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Show only first 3 files
        print(f'{subindent}{file}')
    if len(files) > 3:
        print(f'{subindent}... and {len(files) - 3} more files')

## 2️⃣ Load and Explore Dataset

### Download and Prepare Data
For this example, we'll use the Microsoft Cats vs Dogs dataset.

In [ ]:
# Import core libraries
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

# Deep Learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Image processing
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# Metrics and evaluation
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    roc_curve,
    auc,
    roc_auc_score
)

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 1️⃣ Import Required Libraries

# 🖼️ Image Classifier com Deep Learning
## Classificação de Gatos vs Cães usando CNN

Este notebook implementa um classificador de imagens usando Redes Neurais Convolucionais (CNN) com TensorFlow/Keras.

### 📊 Objetivos
- ✅ Manipulação e preprocessamento de imagens
- ✅ Data Augmentation para melhorar robustez
- ✅ Construção de CNN personalizada
- ✅ Treinamento com monitoramento de overfitting
- ✅ Avaliação com métricas diversas
- ✅ Comparação com Transfer Learning